In [1]:
%load_ext sql
%sql sqlite:///complaint.db
import time
%sql drop index if exists helpful_index;
%sql analyze

Done.
Done.


/Users/tarabalakrishnan/.local/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/tarabalakrishnan/.local/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


[]

Let's play with the [consumer complaint database](https://catalog.data.gov/dataset/consumer-complaint-database) from data.gov

In [ ]:
%sql select count(*) from complaints;

In [ ]:
%sql select * from complaints limit 5;

The syntax to create a view is below
> create index (index_name) on (table)(attributes)

In [ ]:
%%sql drop index if exists zip_index;
create index zip_index on complaints(zip_code);
analyze

In [ ]:
%%sql
select zip_code, count(*) as c 
from complaints group by zip_code 
order by c desc limit 5;

In [ ]:
%%sql select product, count(*)
from complaints c1 where ZIP_CODE = '48382'
group by product;

Let's get down to business... Indexing complainers!

For pedagogical purposes, we'll use _likelihood_ function:
  * likelihood(X,Y) does not change X's value 
  * instead it tells the DBMS the fraction of calls on which we believe X holds.
      * 0.0 is lowest, 1.0 is highest.
  * It's a form of _hint_ for the optimizer

In [ ]:
%%sql 
explain query plan 
select product, count(*) from complaints c1 
where likelihood(ZIP_CODE = '48383' or ZIP_CODE='44444', 1.0)
group by product;

In [ ]:
%%sql 
explain query plan 
select product, count(*) from complaints c1 
where likelihood(ZIP_CODE = '48383' or ZIP_CODE='44444', 0.0)
group by product;

So... does it make a difference in execution time? 

... Well, some it's a small database in memory...

In [ ]:
start_scan = time.time()
%sql select product, count(*) from complaints c1 where likelihood(ZIP_CODE = '48382' or ZIP_CODE='44444', 1.0) group by product;
scan_time = time.time() - start_scan

start_index = time.time()
%sql select product, count(*) from complaints c1 where likelihood(ZIP_CODE = '48382' or ZIP_CODE='44444', 0.0) group by product;
index_time  = time.time() - start_index
print("time for scan={0} time for index={1}".format(scan_time, index_time))
print("\t So about {0} times faster".format(scan_time/index_time))

<center>Covering Indexes</center>

We say that an index is _covering_ if all the needed columns are in the index.
 * Needed columns is the union of those in WHERE _and_ SELECT.
 * Means that the query can be answere using the index--without reading the table!

In [ ]:
%%sql drop index if exists helpful_index;
create index helpful_index on 
complaints(zip_code, product, complaint_id);

Which plan happens for this query?
> select product, count(*)<br>
> from complaints <br>
> where likelihood(ZIP_CODE = '48382', 1.0)
> <br>group by product;

NB: Using zip_index, this would be a scan!

In [ ]:
%%sql
explain query plan select product, count(*) 
from complaints c1 
where likelihood(ZIP_CODE = '48382' or ZIP_CODE='44444', 1.0) group by product;

* The index is _smaller_ in absolute data size.
* The DBMS knows this, and it picks the right index!

Summary
-------

* This notebook walks through index creation 
* That index selection is done in a cost based way.
* Introduced the notion of covering index.
* Demonstrated that not only keys to answer query, but attributes returned important component of selection. 